In [ ]:
# default_exp runner

In [ ]:
import fire
from pathlib import Path
from datetime import datetime
from pathlib import Path
from functools import partial
from tegrastatsjson.command_grabber import *
from tegrastatsjson.parser import *

# Tegrastats callback
reveices the line, parses the data and adds a datetime timestamp for the sample
note, the timestamp is when the callback is called and not when the actual sample is received from tegrastats. but this timelag might not be an issue.

In [ ]:
def tegrastats_callback(tegra_line, filename='/var/output/tegra.json'):
    timestamp = str(datetime.now())
    d = ParseTegrastats.parse_line(line)
    d['timestamp'] = timestamp
    filename = filename
    with open(filename, 'a') as f:
        json.dump(d, f)
        f.write('\n')

In [ ]:

def tegrastatsjson(log_file, interval=1000,  add_date_to_logfile=True):
    """ interval in ms, logfile, logfile, path to the file to be logged. 
    Parameters
        interval: in ms, (same as for tegrastats)
        logfile: path to filename to log to (same as for tegrastats)
        add_date_to_logfile: adds a timestamp to the log file to easier distinguish between different runs on the jetson
        
    """
    p = Path(log_file)
    if add_date_to_logfile:
        fname = p.stem+ datetime.now().strftime("-%Y-%m-%d %H:%M:%S") + p.suffix
        p = p.parent / fname
        
    tegrastats_c = partial(tegrastats_callback, filename=str(p))
    args = ['sh', '-c', f"tegrastats --interval {interval}"]
    
    task = CommandGrabber(args, tegrastats_c)
    task.wait_until_finished()
    return
    